In [1]:
#Rating Product & Sorting Reviews in Amazon

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import math
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
df = pd.read_csv('amazon_review.csv')

In [4]:
df.shape

(4915, 12)

In [5]:
df.dtypes

reviewerID         object
asin               object
reviewerName       object
helpful            object
reviewText         object
overall           float64
summary            object
unixReviewTime      int64
reviewTime         object
day_diff            int64
helpful_yes         int64
total_vote          int64
dtype: object

In [6]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [7]:
df.tail()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
4910,A2LBMKXRM5H2W9,B007WTAJTO,"ZM ""J""","[0, 0]",I bought this Sandisk 16GB Class 10 to use wit...,1.00000,Do not waste your money.,1374537600,2013-07-23,503,0,0
4911,ALGDLRUI1ZPCS,B007WTAJTO,Zo,"[0, 0]",Used this for extending the capabilities of my...,5.00000,Great item!,1377129600,2013-08-22,473,0,0
4912,A2MR1NI0ENW2AD,B007WTAJTO,Z S Liske,"[0, 0]",Great card that is very fast and reliable. It ...,5.00000,Fast and reliable memory card,1396224000,2014-03-31,252,0,0
4913,A37E6P3DSO9QJD,B007WTAJTO,Z Taylor,"[0, 0]",Good amount of space for the stuff I want to d...,5.00000,Great little card,1379289600,2013-09-16,448,0,0
4914,A8KGFTFQ86IBR,B007WTAJTO,Zza,"[0, 0]",I've heard bad things about this 64gb Micro SD...,5.00000,So far so good.,1388620800,2014-02-01,310,0,0


In [8]:
df.sample(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
4727,A59DOB4UY2K9T,B007WTAJTO,villapoo,"[2, 2]",Bought this for Samsung Galaxy S3 in Fall 2012...,3.00000,Awesome when not defective,1368403200,2013-05-13,574,2,2
2648,ATOPUFCOM0M55,B007WTAJTO,Kenneth Kirkland,"[0, 0]",its geniune and performs as expected in my ASU...,5.00000,Does whatits supposed to,1353369600,2012-11-20,748,0,0
2810,A150UYJPK6KBKO,B007WTAJTO,LeDerrius,"[0, 0]",Don't really use this for speed tests or anyth...,4.00000,Nice,1388448000,2013-12-31,342,0,0
2740,A23FM0S3J3JYAL,B007WTAJTO,KRISTOFFER FEATHERINGHAM,"[0, 0]",Had had this for about 6 months now and it's i...,5.00000,Simple and easy...,1389052800,2014-07-01,160,0,0
4354,AXHPIBNP15GL8,B007WTAJTO,Steveotech,"[1, 2]",I bought this on October 9th and didn't start ...,1.00000,Time to shop Kingston,1354665600,2012-05-12,940,1,2


In [9]:
df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime', 'day_diff', 'helpful_yes', 'total_vote'], dtype='object')

In [10]:
df.nunique()

reviewerID        4915
asin                 1
reviewerName      4594
helpful             42
reviewText        4912
overall              5
summary           3885
unixReviewTime     690
reviewTime         690
day_diff           690
helpful_yes         23
total_vote          26
dtype: int64

In [11]:
df.isnull().sum()

reviewerID        0
asin              0
reviewerName      1
helpful           0
reviewText        1
overall           0
summary           0
unixReviewTime    0
reviewTime        0
day_diff          0
helpful_yes       0
total_vote        0
dtype: int64

In [12]:
print(df.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

                    count             mean            std              min               0%               5%              50%              95%              99%             100%              max
overall        4915.00000          4.58759        0.99685          1.00000          1.00000          2.00000          5.00000          5.00000          5.00000          5.00000          5.00000
unixReviewTime 4915.00000 1379465001.66836 15818574.32275 1339200000.00000 1339200000.00000 1354492800.00000 1381276800.00000 1403308800.00000 1404950400.00000 1406073600.00000 1406073600.00000
day_diff       4915.00000        437.36704      209.43987          1.00000          1.00000         98.00000        431.00000        748.00000        943.00000       1064.00000       1064.00000
helpful_yes    4915.00000          1.31109       41.61916          0.00000          0.00000          0.00000          0.00000          1.00000          3.00000       1952.00000       1952.00000
total_vote     4915.00000     

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [14]:
df['overall'].mean()

4.587589013224822

In [15]:
df['reviewTime'] = pd.to_datetime(df['reviewTime'])

In [16]:
today_date = df.reviewTime.max()

In [17]:
df['diff_time'] = (today_date - df["reviewTime"]).dt.days

In [18]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,diff_time
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0,137
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0,714
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,381
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0,512


In [19]:
df['diff_time'].quantile([0, 0.1, 0.25, 0.5, 0.75, 0.95, 1])

0.00000      0.00000
0.10000    166.00000
0.25000    280.00000
0.50000    430.00000
0.75000    600.00000
0.95000    747.00000
1.00000   1063.00000
Name: diff_time, dtype: float64

In [20]:
def time_based_weighted_average(dataframe, w1=30, w2=28, w3=26, w4=16):
    return dataframe.loc[df["diff_time"] <= 250, "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["diff_time"] > 250) & (dataframe["diff_time"] <= 500), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["diff_time"] > 500) & (dataframe["diff_time"] <= 750), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["diff_time"] > 750), "overall"].mean() * w4 / 100

In [21]:
time_based_weighted_average(df)

4.576037285044267

In [22]:
df.loc[df["diff_time"] <= 250, "overall"].mean()

4.684106614017769

In [23]:
df.loc[(df["diff_time"] > 250) & (df["diff_time"] <= 500), "overall"].mean()

4.624681285058643

In [24]:
df.loc[(df["diff_time"] > 500) & (df["diff_time"] <= 750), "overall"].mean()

4.5155607751027595

In [25]:
df.loc[df["diff_time"] > 750, "overall"].mean()

4.38655462184874

In [26]:
df["helpful_no"] = df["total_vote"] - df["helpful_yes"]

In [27]:
df.sample(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,diff_time,helpful_no
1859,A1VSGGV8TETMYD,B007WTAJTO,Gregg F. Carter,"[0, 0]",Bought this for my ASUS TF700T Transformer tab...,5.00000,Sweet microSD,1359417600,2013-01-29,678,0,0,677,0
4286,A1B1VDFFC2N8PH,B007WTAJTO,S. Rothrock,"[0, 0]","These are excellent SD cards for phones, camer...",5.00000,Great product,1356480000,2012-12-26,712,0,0,711,0
1951,A28C8ZFBA0HSUQ,B007WTAJTO,Heath,"[0, 0]",Works as expected. Love the great price and us...,5.00000,Great memory,1370476800,2013-06-06,550,0,0,549,0
1484,A37FCNU3EY6LZM,B007WTAJTO,D. Woods,"[0, 0]",This mem card is fast and does exactly what is...,5.00000,Great memory card,1368576000,2013-05-15,572,0,0,571,0
3345,A36U8X10T0BFVJ,B007WTAJTO,M. Stanley Bubien,"[2, 2]",My review is a little backwards&#8230; but I t...,5.00000,Watch Price - Great Card Often on Sale,1388966400,2014-06-01,190,2,2,189,0
3487,A1LJMQNR9C9DEW,B007WTAJTO,odey,"[0, 0]",excelent,5.00000,Five Stars,1404691200,2014-07-07,154,0,0,153,0
3325,A2WH3SN2MODL54,B007WTAJTO,"MR. MAV ""Maverickccs""","[1, 3]",The only reason I give one star is that I cant...,1.00000,LOST ALL MY FILES IN MY S3,1398729600,2014-04-29,223,1,3,222,2
3966,A1DRKQ4Q5MB6RJ,B007WTAJTO,"Roy Samuel ""The Real Deal""","[0, 0]",Lives up the billing of very fast data transfe...,5.00000,80GB on my Note 2? I'll take it!,1355184000,2012-11-12,756,0,0,755,0
4859,A21HP6M30Q5PAA,B007WTAJTO,W.T. HAMPTON,"[0, 0]",With the adapter I can easily use it in my cam...,5.00000,VERSATILE !,1395619200,2014-03-24,259,0,0,258,0
993,A34ET2WSQAFDG1,B007WTAJTO,"Clay Sloan ""Chocolater""","[0, 0]",I mean it's a 64gb memory card. The price was ...,5.00000,"Big memory card, good price",1384992000,2013-11-21,382,0,0,381,0


In [28]:
df = df[["overall", "reviewTime", "day_diff", "helpful_yes", "helpful_no","total_vote"]]
df.head()

,overall,reviewTime,day_diff,helpful_yes,helpful_no,total_vote
0,4.00000,2014-07-23,138,0,0,0
1,5.00000,2013-10-25,409,0,0,0
2,4.00000,2012-12-23,715,0,0,0
3,5.00000,2013-11-21,382,0,0,0
4,5.00000,2013-07-13,513,0,0,0


In [29]:
def score_up_down_diff(up, down):
    return up - down

In [30]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [31]:
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Wilson Lower Bound Score hesapla

    - Bernoulli parametresi p için hesaplanacak güven aralığının alt sınırı WLB skoru olarak kabul edilir.
    - Hesaplanacak skor ürün sıralaması için kullanılır.
    - Not:
    Eğer skorlar 1-5 arasıdaysa 1-3 negatif, 4-5 pozitif olarak işaretlenir ve bernoulli'ye uygun hale getirilebilir.
    Bu beraberinde bazı problemleri de getirir. Bu sebeple bayesian average rating yapmak gerekir.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [32]:
df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x["helpful_yes"], x["helpful_no"]), axis=1)

In [33]:
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)

In [34]:
df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)

In [35]:
df.head()

,overall,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,4.00000,2014-07-23,138,0,0,0,0,0.00000,0.00000
1,5.00000,2013-10-25,409,0,0,0,0,0.00000,0.00000
2,4.00000,2012-12-23,715,0,0,0,0,0.00000,0.00000
3,5.00000,2013-11-21,382,0,0,0,0,0.00000,0.00000
4,5.00000,2013-07-13,513,0,0,0,0,0.00000,0.00000


In [36]:
df.sort_values("wilson_lower_bound", ascending=False).head(20)

,overall,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,5.00000,2013-01-05,702,1952,68,2020,1884,0.96634,0.95754
3449,5.00000,2012-09-26,803,1428,77,1505,1351,0.94884,0.93652
4212,1.00000,2013-05-08,579,1568,126,1694,1442,0.92562,0.91214
317,1.00000,2012-02-09,1033,422,73,495,349,0.85253,0.81858
4672,5.00000,2014-07-03,158,45,4,49,41,0.91837,0.80811
1835,5.00000,2014-02-28,283,60,8,68,52,0.88235,0.78465
3981,5.00000,2012-10-22,777,112,27,139,85,0.80576,0.73214
3807,3.00000,2013-02-27,649,22,3,25,19,0.88000,0.70044
4306,5.00000,2012-09-06,823,51,14,65,37,0.78462,0.67033
4596,1.00000,2012-09-22,807,82,27,109,55,0.75229,0.66359
